In [95]:
import plotly.graph_objs as go
import plotly.express as px
import matplotlib as plt
import pandas as pd
import numpy as np
import seaborn as sns
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import GridspecLayout
import ipywidgets as widgets

In [96]:
layout = go.Layout(
        font=dict(
        family=""""Lato, "Helvetica Neue", Helvetica, Arial, "Liberation Sans", sans-serif""",
        ),
        title=dict(font=dict(size=24)),
        newshape_label_padding=8,
        margin_pad=5,
        legend=dict(font=dict(size=14)),
    )



In [97]:
import pandas as pd
df_per_product = pd.read_excel('dataset1.xls', sheet_name=2, skiprows=2, nrows=70, usecols="A, F:L")
df_per_product = df_per_product.rename(columns={'LUC': 'Land Use Change', 'Packging': 'Packaging'})

df = df_per_product[0:43].copy()
df['Total'] = df.iloc[:, 1:].sum(axis=1)
# display(df)
df = df.sort_values(by='Total')

# display(df.Product)

df_melted = pd.melt(df, id_vars='Product', value_vars=['Land Use Change', 'Feed', 'Farm', 'Processing', 'Transport', 'Packaging', 'Retail'], var_name='Stage', value_name='Emissions')

# Reshape the DataFrame into a "long" format
df_melted = pd.melt(df, id_vars='Product', value_vars=['Land Use Change', 'Feed', 'Farm', 'Processing', 'Transport', 'Packaging', 'Retail'], var_name='Stage', value_name='Emissions')


In [98]:
df_cat = pd.read_excel('dataset1.xls', sheet_name=2, skiprows=2, nrows=43, usecols="A:B, F:L")
df_cat = df_cat.rename(columns={'LUC': 'Land Use Change', 'Packging': 'Packaging', "Food and Waste ('000 t, 2009-11 avg.)": 'Amount produced'})
df_cat['Total emissions'] = df_cat.iloc[:, 2:].sum(axis=1)
df_cat['Total impact'] = df_cat['Total emissions'] * df_cat['Amount produced']

def add_label(name, df, figure):
    figure.add_annotation(
        x=np.log10(df.loc[df['Product'] == name, 'Amount produced'].iloc[0]),
        y=df.loc[df['Product'] == name, 'Total emissions'].iloc[0],
        text=name,
        showarrow=True,
        yshift=5
    )

def add_labels(names, df, figure):
    for name in names:
        add_label(name, df, figure)

# assigns categories
df_cat['Type'] = 'Crops'
df_cat.loc[33:37, 'Type'] = 'Meat'
df_cat.loc[38:40, 'Type'] = 'Dairy'
df_cat.loc[41:42, 'Type'] = 'Seafood'

fig = px.scatter(
    df_cat, 
    x="Amount produced",
    y="Total emissions",
    color="Type",
    log_x=True,
    height=500,
    # text='Product',
    # log_y=True,
    # size="Total impact",
    size_max=60,
    hover_data=['Product'],
    title="GHG impact and global production"
    )

fig.update_traces(marker_size=10)

highlighted = ['Bovine Meat (beef herd)', 'Cheese', 'Lamb & Mutton', 'Dark Chocolate', 'Fish (farmed)', 'Eggs', 'Poultry Meat', 'Milk', 'Potatoes', 'Milk', 'Rice', 'Soymilk', 'Nuts', 'Bananas', 'Apples']

fig.update_layout(yaxis_title='Emissions (kg CO2eq / kg)', xaxis_title='Global production (tonnes)')
add_labels(highlighted, df_cat, fig)




fig.show()

# display(df_cat)


Transport costs vary

In [99]:
df_transport = df.sort_values(by='Transport')

fig1 = px.bar(df_transport,
    y='Product',
    x='Transport',
    height=800,
    title="Transport emissions per product")

fig1.show()

But mean little in the grand scheme of things

In [100]:

standard_discrete = px.colors.qualitative.T10

print(sns.color_palette("tab10").as_hex())
# Create a horizontal stacked bar plot using plotly.express
fig = px.bar(df_melted,
             y='Product',
             x='Emissions',
             color='Stage',
             width=1000,
             height=800,
             color_discrete_sequence=standard_discrete,
             title="Greenhouse gas emissions per food type over the supply chain")

fig.update_traces()

fig.update_layout(barmode='relative')

fig.add_scatter(y=df['Product'], x=df['Total'], mode='markers', name='Total', marker=dict(symbol='line-ns-open', color="black", size=7))

fig.update_layout(layout,        
        legend=dict(
        orientation="h",
        yanchor="top",
        y=1.05,
        xanchor="left",
        x=-0.06,
    )
    )


fig.update_traces(patch={"opacity": 1}, selector={"legendgroup": "Transport"})

fig.show()




['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']


In [101]:

standard_discrete = px.colors.qualitative.T10

print(sns.color_palette("tab10").as_hex())
# Create a horizontal stacked bar plot using plotly.express
fig = px.bar(df_melted,
             y='Product',
             x='Emissions',
             color='Stage',
             width=1000,
             height=800,
             color_discrete_sequence=standard_discrete,
             title="Greenhouse gas emissions per food type over the supply chain")

fig.update_traces(patch={"opacity": 0.1})

fig.update_layout(barmode='relative')

# fig.add_scatter(y=df['Product'], x=df['Total'], mode='markers', name='Total', marker=dict(symbol='line-ns-open', color="black", size=7))

fig.update_layout(layout,        
        legend=dict(
        orientation="h",
        yanchor="top",
        y=1.05,
        xanchor="left",
        x=-0.06,
    )
    )


fig.update_traces(patch={"opacity": 1}, width=1, selector={"legendgroup": "Transport"})

fig.show()


['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']


An animation will be added here to transition from the prior graph to this one

In [102]:
df2 = df_per_product.iloc[55]
display(df2)
fig = px.pie(df2, values=df2.values[1:], names=df2.index[1:], hole=.3, height=600, title='Percentage of total GHG emissions by part of supply chain')
fig.update_traces(textposition='outside', textinfo='label+percent', marker=dict(colors=standard_discrete, line=dict(color='#000000', width=2)), showlegend=False)
fig.show()

Product            Food Total (M ha; Gg; km3)
Land Use Change                    2379469.67
Feed                              1098394.985
Farm                              7463342.423
Processing                         604297.515
Transport                          801403.803
Packaging                           626870.68
Retail                             394202.635
Name: 55, dtype: object